In [1]:
import openmmtools
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
from sys import stdout
import numpy as np
from importlib import reload

In [2]:
pdb = app.PDBFile('liquid.pdb')
forcefield = app.ForceField('./dms.xml')

system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, 
    nonbondedCutoff=1.0*unit.nanometers, rigidWater=True, 
    ewaldErrorTolerance=0.0005)

integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 
    1.0*unit.femtoseconds)

system.addForce(mm.MonteCarloBarostat(1*unit.atmospheres, 300*unit.kelvin, 25))

platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [3]:
reload(openmmtools.alchemy)
factory = openmmtools.alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False, split_alchemical_forces = True, alchemical_pme_treatment = 'exact')
reference_system = system

alchemical_region_0 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [0,1,2,3,4,5,6,7,8], alchemical_angles = [6], alchemical_torsions = [0,1,2,3,4,5], name='zero')
alchemical_region_1 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [9,10,11,12,13,14,15,16,17], alchemical_torsions = [6,7,8,9,10,11], name='one')
alchemical_region_2 = openmmtools.alchemy.AlchemicalRegion(alchemical_atoms = [18,19,20,21,22,23,24,25,26], alchemical_angles = [32],  name='two')

alchemical_system_in = factory.create_alchemical_system(reference_system, alchemical_regions = [alchemical_region_0, alchemical_region_1, alchemical_region_2])

alchemical_state_zero = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'zero')
alchemical_state_one = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'one')
alchemical_state_two = openmmtools.alchemy.AlchemicalState.from_system(alchemical_system_in, region_name = 'two')

Using 3 alchemical regions


In [4]:
TS = openmmtools.states.ThermodynamicState(alchemical_system_in, temperature=300*unit.kelvin, pressure=1*unit.bar)
composable_states = [alchemical_state_zero, alchemical_state_one, alchemical_state_two]
compound_state = openmmtools.states.CompoundThermodynamicState(
            thermodynamic_state=TS, composable_states=composable_states)

In [5]:
reload(openmmtools.alchemy)

compound_state._composable_states[0].lambda_sterics_zero = 1.0
compound_state._composable_states[0].lambda_electrostatics_zero = 1.0

compound_state._composable_states[1].lambda_sterics_one = 0.0
compound_state._composable_states[1].lambda_electrostatics_one = 0.0

compound_state._composable_states[2].lambda_sterics_two = 0.0
compound_state._composable_states[2].lambda_electrostatics_two = 0.0

compound_state._composable_states[0].lambda_torsions_zero = 0.0
compound_state._composable_states[1].lambda_torsions_one = 0.0

compound_state._composable_states[0].lambda_angles_zero = 0.0
compound_state._composable_states[2].lambda_angles_two = 0.0

sys = compound_state.get_system()
file = open('DEBUG_system.xml','w')
file.write(mm.XmlSerializer.serialize(sys))
file.close()

simulation = app.Simulation(pdb.topology, alchemical_system_in, integrator, platform, 
    properties)
compound_state.apply_to_context(simulation.context)

In [6]:
simulation.context.setPositions(pdb.positions)
'''
for param in simulation.context.getParameters():
    print(param)
    print(simulation.context.getParameter(param))
'''
print('Minimizing...')
#simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(100)
simulation.reporters.append(app.DCDReporter('trajectoryM.dcd', 1000))
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True, 
    potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
    speed=True, totalSteps=50000, separator='\t'))

print('Running Production...')
simulation.step(50000)
print('Done!')

Minimizing...
Equilibrating...
Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
2.0%	1000	4507.257027256577	354.0954292503473	0	--
4.0%	2000	3946.0609561432284	323.2921676835446	102	0:40
6.0%	3000	3557.355271076406	311.3061835302581	103	0:39
8.0%	4000	3023.3132342036106	309.5627642656892	104	0:38
10.0%	5000	3196.5257949976576	302.7229247105115	104	0:37
12.0%	6000	3076.934498643297	305.9884552595506	102	0:37
14.0%	7000	3117.868146181868	302.3120448499768	102	0:36
16.0%	8000	3126.9064519444864	302.0908671745753	102	0:35
18.0%	9000	3273.79154534965	295.8371652846034	103	0:34
20.0%	10000	3137.962350372127	302.4107052490276	103	0:33
22.0%	11000	3241.78179343794	302.70425896123993	103	0:32
24.0%	12000	3059.347999671173	303.6568480344614	103	0:31
26.0%	13000	3357.0250568595357	296.38142270158266	103	0:30
28.0%	14000	3047.0753923448365	305.30800427463697	103	0:30
30.0%	15000	3005.46708509358	296.86099320669695	103	0:2

KeyboardInterrupt: 